In [1]:
import torch

In [192]:
def absmax_quantize_i8(A:torch.Tensor):
    absmax = torch.max(torch.abs(A))
    A_i8 = ((A * 127) / absmax).to(torch.int8)
    return A_i8, A_i8.to(torch.float32) * absmax / 127

def zeropoint_quantize_i8(A:torch.Tensor):
    r = torch.max(A) - torch.min(A)
    r = 1 if r == 0 else r
    scale = 255 / r
    zeropoint = (-scale * torch.min(A) - 128)
    A_i8 =  (A * scale + zeropoint).round().to(torch.int8)
    return A_i8, (A_i8 - zeropoint) / scale

def zeropoint_quantize(X):
    # Calculate value range (denominator)
    x_range = torch.max(X) - torch.min(X)
    x_range = 1 if x_range == 0 else x_range

    # Calculate scale
    scale = 255 / x_range

    # Shift by zero-point
    zeropoint = (-scale * torch.min(X) - 128).round()
    # Scale and round the inputs
    X_quant = torch.clip((X * scale + zeropoint).round(), -128, 127)

    # Dequantize
    X_dequant = (X_quant - zeropoint) / scale

    return X_quant.to(torch.int8), X_dequant

In [202]:
A = torch.randn(30,30)
A_i8, A_recon = absmax_quantize_i8(A)
torch.sum(torch.abs(A - A_recon))

tensor(11.8051)

In [203]:
A_i8, A_recon = zeropoint_quantize_i8(A)
torch.sum(torch.abs(A - A_recon))

tensor(5.7245)

In [204]:
A_i8, A_recon = zeropoint_quantize(A)
torch.sum(torch.abs(A - A_recon))

tensor(5.8415)